<h1>Which city to visit for Asian food?</h1>

<b>Introduction</b>

Let’s say you were planning a road trip around the United States and want to get the best Asian foods. You would plan to visit the places with the highest density of Asian restaurants. The problem we want to solve is to analyze Asian restaurants locations in the major cities and find the best place to enjoy Asian foods. Our target audiences are people who love Asian foods and are planning a trip around it.

In [4]:
!pip install folium

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 93 kB 3.2 MB/s  eta 0:00:01
Libraries imported.


In [6]:
CLIENT_ID = '0VSRILUAGUITUMIHGE1HHSU4VGYZNPRH15XMT30S0DFUK4UA' # your Foursquare ID
CLIENT_SECRET = 'SVIKDDB3CBAOKHO1X41CDGNF12FNAWG2XKCLKXHS5PH2LYYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 0VSRILUAGUITUMIHGE1HHSU4VGYZNPRH15XMT30S0DFUK4UA


In [8]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Los Angeles, CA', 'Chicago, IL', 'Houston, TX', 'Phoenix, AZ']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # Asian Restaurant FourSquare CATEGORY ID
    results[city] = requests.get(url).json()

In [10]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


In [11]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Asian Restaurant in {city} = ", results[city]['response']['totalResults'])

Total number of Asian Restaurant in New York, NY =  277
Total number of Asian Restaurant in Los Angeles, CA =  237
Total number of Asian Restaurant in Chicago, IL =  217
Total number of Asian Restaurant in Houston, TX =  197
Total number of Asian Restaurant in Phoenix, AZ =  180


In [12]:
maps[cities[0]]

In [13]:
maps[cities[1]]

In [14]:
maps[cities[2]]

In [15]:
maps[cities[3]]

In [16]:
maps[cities[4]]

We can see that New York isthe most dense city with Asian Restaurants. 

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [17]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.023390692599065996
Los Angeles, CA
Mean Distance from Mean coordinates
0.08253135036564828
Chicago, IL
Mean Distance from Mean coordinates
0.05942143986316826
Houston, TX
Mean Distance from Mean coordinates
0.13902530270291108
Phoenix, AZ
Mean Distance from Mean coordinates
0.09673026204088916


In [18]:
maps[cities[0]]

In [19]:
maps[cities[1]]

In [20]:
maps[cities[2]]

In [21]:
maps[cities[3]]

In [22]:
maps[cities[4]]

We now see that New York is his best option. Our traveler's best option would be to book a hotel near that mean coordinate to surround himself with the 100 Asian Restaurant there!!